<a href="https://colab.research.google.com/github/DianeMack22/econ8310-assignment3/blob/main/Test_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# -----------------------------
# 11. Load and Use the Model
# -----------------------------
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import struct
import numpy as np
from torch.utils.data import Dataset

# -----------------------------
# CustomMNIST Dataset Class
# -----------------------------
class CustomMNIST(Dataset):
    def __init__(self, image_path, label_path, transform=None):
        self.images = self._read_images(image_path)
        self.labels = self._read_labels(label_path)
        self.transform = transform

    def _read_images(self, path):
        with open(path, 'rb') as f:
            magic, num, rows, cols = struct.unpack(">IIII", f.read(16))
            image_data = np.frombuffer(f.read(), dtype=np.uint8)
            images = image_data.reshape((num, rows, cols))
        return images

    def _read_labels(self, path):
        with open(path, 'rb') as f:
            magic, num = struct.unpack(">II", f.read(8))
            label_data = np.frombuffer(f.read(), dtype=np.uint8)
        return label_data

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        image = torch.tensor(image, dtype=torch.float32).unsqueeze(0) / 255.0
        return image, label

# -----------------------------
# Model Architecture
# -----------------------------
import torch.nn.functional as F

class FirstNet(nn.Module):
    def __init__(self):
        super(FirstNet, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_model = nn.Sequential(
            nn.LazyLinear(10),
        )

    def forward(self, x):
        x = self.flatten(x)
        output = self.linear_relu_model(x)
        return output

# -----------------------------
# Load Model and Evaluate
# -----------------------------
def evaluate():
    # Load test dataset
    test_dataset = CustomMNIST(
        image_path="data/t10k-images-idx3-ubyte",
        label_path="data/t10k-labels-idx1-ubyte"
    )
    test_loader = DataLoader(test_dataset, batch_size=64)

    # Recreate model and optimizer
    model = FirstNet()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

    # Load weights
    checkpoint = torch.load("model1.pt")
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    # Evaluate
    correct = 0
    total = 0
    with torch.no_grad():
        for X, y in test_loader:
            outputs = model(X)
            _, predicted = torch.max(outputs.data, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()

    print(f"Accuracy of the loaded model: {100 * correct / total:.2f}%")

if __name__ == "__main__":
    evaluate()

FileNotFoundError: [Errno 2] No such file or directory: 'data/t10k-images-idx3-ubyte'